In [1]:
import pandas as pd
import re

In [2]:
dicionario_fases = { #Informações extraidas do doc FASES_PROD
    1 :'ENGENHARIA',
    5 :'PRENSA',
    10 :'CORTE',
    12 :'FITA MANUAL',
    15 :'CNC',
    20 :'MAQUINACAO',
    25 :'MAQUINACAO III',
    26 :'CNC',
    30 :'ACABAMENTO',
    34 :'MAQ MANUAL',
    35 :'MAQUINACAO IV',
    37 :'MONTAGEM INTERNA I',
    38 :'SOLDA',
    39 :'FITA MANUAL',
    40 :'PINTURA',
    41 :'COMPRAR',
    45 :'MAQUINACAO V',
    50 :'ACABAMENTO II',
    60 :'MONTAGEM INTERNA II',
    70 :'EMBALAGEM',
    75 :'EXPEDICAO - PROD FINAL',
    80 :'CHECKLIST EXTERNO MONT',
    85 :'CHECKLIST EXTERNO FIX PAREDE',
    90 :'ENTREGA PRODUTO FINAL',
    99 :'MANUTENCAO'
}

dicionario_materiais = {
    "100.10": "MDF",
    "100.20": "COMPENSADOS",
    "100.30": "AGLOMERADOS",
    "100.40": "SARRAFEADOS",
    "100.50": "MACICOS",
    "100.60": "MDP",
    "100.70": "LAMINADAS",
    "100.99": "SUCATA DE MDF",
    "101.10": "BATISMO TREINAMENTO",
    "150.10": "LAMINAS DE MADEIRA",
    "150.15": "PET",
    "150.20": "FORMICAS",
    "150.25": "PVC",
    "150.30": "FITAS DE BORDA",
    "150.35": "COUROS",
    "150.40": "BORRACHAS",
    "150.45": "TECIDOS",
    "150.50": "ESTOFADOS",
    "150.55": "POLIMEROS",
    "150.99": "SUCATA DE REVESTIMENTOS",
    "200.10": "TINTAS",
    "200.15": "SOLVENTES",
    "200.20": "VERNIZES",
    "200.25": "COLAS",
    "200.30": "CERAS",
    "200.35": "ESPUMAS",
    "200.40": "LUBRIFICANTES",
    "200.45": "APLICADORES",
    "250.10": "LIXAS",
    "250.20": "ESCOVAS",
    "300.10": "VIDROS",
    "300.20": "ESPELHOS",
    "300.30": "ACRILICOS",
    "350.10": "BARRA",
    "350.20": "TUBO",
    "350.30": "CHAPA",
    "350.40": "COMPONENTES",
    "400.10": "PLASTICO",
    "400.20": "PAPELAO",
    "400.30": "MADEIRA",
    "400.40": "ISOPOR",
    "400.50": "CORDAS",
    "450.10": "FRESAS",
    "450.20": "SERRAS",
    "450.30": "BROCAS",
    "450.40": "AGREGADOS",
    "450.50": "FACAS",
    "450.60": "MANUAIS",
    "500.10": "PARAFUSOS",
    "500.15": "PREGOS",
    "500.20": "PINOS",
    "500.25": "GRAMPOS",
    "500.30": "EMBUTIDOS",
    "500.35": "BUCHAS",
    "500.40": "PORCAS",
    "500.45": "ARRUELAS",
    "500.50": "REBITES",
    "530.40": "PORCAS",
    "550.10": "DOBRADICAS",
    "550.15": "CORREDICAS",
    "550.20": "BASCULANTES",
    "550.25": "DESLIZANTES",
    "550.30": "FECHADURAS",
    "550.35": "SAPATAS",
    "550.40": "DESEMPENADORES",
    "550.45": "TRINCOS",
    "550.50": "PRATOS",
    "550.55": "FECHOS",
    "550.60": "RODIZIOS",
    "550.65": "SUPORTES",
    "600.10": "PUXADORES",
    "600.20": "ACABAMENTOS",
    "600.30": "GERAIS",
    "600.40": "ILUMINACAO",
    "600.50": "PES"
}

tipos_material = ['barra', 'tubo', 'perfil', 'chapa']
materiais = list(dicionario_materiais.values())
materiais = [i.lower() for i in materiais]

esq01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : ['aluminio', 'alum'],
    'materiais_proibidos' : [x for x in materiais if x not in ['aluminio', 'alum']],

    'tipo_materiais_proibidos' : ['chapa'],
    'tipo_materiais_permitidos' : [x for x in tipos_material if x not in ['chapa']]
}

esq02 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

esq03 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : 90,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos', 'formicas', 'acrilicos', 'madeira', 'laminas de madeira'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos', 'formicas', 'acrilicos', 'madeira', 'laminas de madeira']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

esq04 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : materiais,  #Aqui sabemos que podemser imputados apenas materiais de metal, mas não conseguimos tirar essa informação somete a partir do doc.
    'materiais_proibidos' : [],

    'tipo_materiais_proibidos' : ['chapa'],
    'tipo_materiais_permitidos' : [x for x in tipos_material if x not in ['chapa']]
}

nes01 = {
    'dimensao_minima' : 130,
    'dimensao_maxima' : None,

    'espessura_minima' : 12,
    'espessura_maxima' : 25,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

sec01 = {
    'dimensao_minima' : 70,
    'dimensao_maxima' : None,

    'espessura_minima' : 3,
    'espessura_maxima' : 30,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

sec02 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : 69,

    'espessura_minima' : 3,
    'espessura_maxima' : 30,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

ser01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

ser02 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos', 'madeira', 'laminas de madeira'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos', 'madeira', 'laminas de madeira']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

col01 = {
    'dimensao_minima' : 69,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : 60,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados', 'madeira', 'laminas de madeira'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados', 'madeira', 'laminas de madeira']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

col02 = {
    'dimensao_minima' : 69,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : 60,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados', 'madeira', 'laminas de madeira'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados', 'madeira', 'laminas de madeira']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

col03 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : 69,

    'espessura_minima' : None,
    'espessura_maxima' : 60,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados', 'madeira', 'laminas de madeira','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos'],
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados', 'madeira', 'laminas de madeira','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

mob01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : materiais, #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : [],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in tipos_material],
    'tipo_materiais_permitidos' : tipos_material
}

mob02 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos', 'madeira', 'laminas de madeira'], #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos', 'madeira', 'laminas de madeira']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

pre01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'madeira', 'laminas de madeira'], #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'madeira', 'laminas de madeira']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

pre02 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : 120,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'madeira', 'laminas de madeira'], #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'madeira', 'laminas de madeira']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

maq01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : materiais, #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : [x for x in materiais if x not in materiais],

    'tipo_materiais_proibidos' : ['chapa'],
    'tipo_materiais_permitidos' : [x for x in tipos_material if x not in ['chapa']]
}

cnc01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : 310,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : materiais, #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : [x for x in materiais if x not in materiais],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa','barra']],
    'tipo_materiais_permitidos' : ['chapa', 'barra']
}

fur01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : materiais, #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : [x for x in materiais if x not in materiais],

    'tipo_materiais_proibidos' : ['chapa'],
    'tipo_materiais_permitidos' : [x for x in tipos_material if x not in ['chapa']]
}

fur02 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos', 'madeira', 'laminas de madeira'], #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos', 'madeira', 'laminas de madeira']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

fur03 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos', 'madeira', 'laminas de madeira'], #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : [x for x in materiais if x not in ['mdf', 'mdp', 'compensados','sucata de mdf', 'sucata de revestimentos','formicas', 'acrilicos', 'madeira', 'laminas de madeira']],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in ['chapa']],
    'tipo_materiais_permitidos' : ['chapa']
}

tup01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : [x for x in materiais if x not in ['formicas']], #Aqui encontramos o mesmo problema do metal
    'materiais_proibidos' : ['formicas'],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in tipos_material],
    'tipo_materiais_permitidos' : tipos_material
}

cab01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : None,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : materiais,
    'materiais_proibidos' : [x for x in materiais if x not in materiais],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in tipos_material],
    'tipo_materiais_permitidos' : tipos_material
}

emb01 = {
    'dimensao_minima' : None,
    'dimensao_maxima' : 120,

    'espessura_minima' : None,
    'espessura_maxima' : None,

    'materiais_permitidos' : materiais,
    'materiais_proibidos' : [x for x in materiais if x not in materiais],

    'tipo_materiais_proibidos' : [x for x in tipos_material if x not in tipos_material],
    'tipo_materiais_permitidos' : tipos_material
}

restricoes = {
    'esq01' : esq01,
    'esq02' : esq02,
    'esq03' : esq03,
    'esq04' : esq04,
    'nes01' : nes01,
    'sec01' : sec01,
    'sec02' : sec02,
    'ser01' : sec01,
    'ser02' : ser02,
    'col01' : col01,
    'col02' : col02,
    'col03' : col03,
    'mob01' : mob01,
    'mob02' : mob02,
    'pre01' : pre01,
    'pre02' : pre02,
    'maq01' : maq01,
    'cnc01' : cnc01,
    'fur01' : fur01,
    'fur02' : fur02,
    'fur03' : fur03,
    'tup01' : tup01,
    'cab01' : cab01,
    'emb01' : emb01

}

maquinas_com_restricao = list(restricoes.keys())

In [3]:
class Preprocessamento:
    '''Para essa classe, é buscado modular de maneira específica métodos e atributos que atendem ao contexto do problema.
    Os métodos podem atualizar o dataframe ou apenas retornar uma parte especíifica do mesmo.
        Métodos que iniciem com extrair apenas retornam as colunas desejadas
        Métodos que iniciem com adiciona ou remove irão alterar o estado do dataset inicial'''
    def __init__(self, imol_data):
        self.imol_data = imol_data
        self.df = imol_data.df



# ----------------------------------------------------------------- # funções de preprocessamento
    def limpar(self, colunas_irrelevantes=None):
        '''Aqui, além de limpar os dados nulos, iremos retirar as linhas e colunas que não possuem informação relevante'''
        if colunas_irrelevantes is None:
            colunas_irrelevantes = ['i', 'QTD', 'Soma de Contagem', 'Unnamed: 0']

        self.imol_data.df.dropna(inplace=True)  # Excluindo linhas nulas
        self.imol_data.df = self.imol_data.df.drop(columns=colunas_irrelevantes)  # Excluindo colunas irrelevantes
        self.imol_data.df = self.imol_data.df[~self.imol_data.df['MATERIAL'].astype(str).str.contains('- -')]  # Removendo as linhas que não contém informação sobre o material
        self.imol_data.df = self.imol_data.df[self.imol_data.df['FLUXO'].str.strip() != '-'] #Removendo as linhas que não contém informação sobre o fluxo

    def limpar_descricao(self):
        '''Esse método tem por objetivo retirar as dimensões no campo descrição, visto que as mesmas já estão presentes nas demais colunas'''
        self.imol_data.df['DESCRIÇÃO'] = self.imol_data.df['DESCRIÇÃO'].apply(lambda x: re.sub(r'\b\w*\d\w*\b', '', x))
        self.imol_data.df['DESCRIÇÃO'] = self.imol_data.df['DESCRIÇÃO'].apply(self.ajusta_texto)


    def adicionar_codigos(self, add_prod=True, add_mat=True):
        '''Estaremos trabalhando com os códigos disponibilizados no pdf CODIGO. Onde é possível perceber algumas coisas interessantes. Os códigos descritos são encontrados nas colunas Produto e Material. Os códigos de materiais vão do número 800 em diante. Onde a partirdo 851 até o 892, apenas os 3 primeiros números são informativos conforme o documento.
        Já os códigos de materiais vão de 100 a 600.'''

        if add_prod:
            self.imol_data.df['COD_PRODUTO'] = self.imol_data.df.apply(self.extrair_codigo_produto, axis=1)
            # self.imol_data.df = self.imol_data.df.drop(columns='PRODUTO') #como após extrairmos o código do produto a coluna produto fica irrelevante iremos excluir
        if add_mat:
            self.imol_data.df['COD_MATERIAL'] = self.imol_data.df['MATERIAL'].apply(lambda x: x[:6])
            self.imol_data.df['MATERIAL_NOME'] = self.imol_data.df['COD_MATERIAL'].apply(lambda x: self.extrair_nome_material(cod_mat=x)) #Vamos adicionar o nome conforme o material empregado, essa informação está no doc

    def ajustar_comp_larg(self):
        '''A ideia dessa função advem de que  comprimento e largura são relativos, e portanto, para padronizar, iremos admitir o campo COMP como sendo sempre o maior lado da peça, ou seja, se larg for maior que comp iremos transpor a informação'''
        self.imol_data.df['COMP'] = self.imol_data.df['COMP'].astype(str).apply(lambda texto : texto.replace(',', '.')) #trazendo os racionais para o padrão float
        self.imol_data.df['LARG'] = self.imol_data.df['LARG'].astype(str).apply(lambda texto : texto.replace(',', '.'))

        condicao = self.imol_data.df['COMP'].astype(float) < self.imol_data.df['LARG'].astype(float)
        self.imol_data.df.loc[condicao, ['COMP', 'LARG']] = self.imol_data.df.loc[condicao, ['LARG', 'COMP']].values

    def adicionar_area(self):
        self.imol_data.df['AREA'] = self.imol_data.df['COMP'].astype(float) * self.imol_data.df['LARG'].astype(float)

    def adicionar_espessura(self):
        '''A espessura é uma informação que será derivada das demais colunas, é possível encontra-la no campo descrição e no campo material. Onde é preciso tomar cuidado, pois não é garantido que teremos a espessura para todos os dados, sendo assim, para dados que não temos informações calcularemos a espessura como -1
            Por padrão para o material, quando encontramos essa informação ela está atrás do sufixo MM
            Já para a descrição, ela está no seguinte formato, largXcompXespessura'''

        self.imol_data.df['MATERIAL'] = self.imol_data.df['MATERIAL'].astype(str).apply(lambda texto : texto.replace(',', '.')) #trazendo ao padrão float

        self.imol_data.df['ESPESSURA'] = self.imol_data.df['MATERIAL'].astype(str).apply(self.extrair_espessura_material) #Retirando espessura do material através da função extrair_espessura_material
        self.imol_data.df['ESPESSURA'] = self.imol_data.df.apply(lambda row: self.extrair_espessura_descricao(row['DESCRIÇÃO']) if row['ESPESSURA'] == -1 else row['ESPESSURA'], axis=1) #retirando espessura da descrição através da função extrair_espessura_descricao somente se já não tiver sido preenchido pelo material

        #Caso a espessura não tenha sido encontrada, mas tenhamos a informação de que comp ou larg=0 tomaremos a espessura como 0
        mascara = (self.imol_data.df['ESPESSURA'] == -1) & ((self.imol_data.df['COMP'].astype(float) == 0) | (self.imol_data.df['LARG'].astype(float) == 0))
        self.imol_data.df.loc[mascara, 'ESPESSURA'] = 0

    def adicionar_fase(self, fase_nome=True, fase_num=True):
        '''Adiciona uma nova coluna ao DataFrame com os nomes das fases correspondentes aos números extraídos da coluna 'FLUXO' '''
        df_fases = self.extrair_fase()
        if fase_num:
            self.imol_data.df['FASE_NUMERO'] = df_fases['FASE_NUMERO']
        if fase_nome:
            self.imol_data.df['FASE_NOME'] = df_fases['FASE_NOME']

        return self.imol_data.df



# ----------------------------------------------------------------- # executar o preprocessamento
    def tratar(self):
        '''Essa função tem por intuito executar todo processamento da classe'''
        self.limpar()
        self.limpar_descricao()
        self.adicionar_espessura()
        self.adicionar_fase()
        self.ajustar_comp_larg()
        self.adicionar_area()
        self.adicionar_codigos()



# ----------------------------------------------------------------- # funções de supoerte para o preprocessamento
    def extrair_codigo_produto(self, linha):
        '''A lógica por trás dos valores é descrita no método adicionar_codigos'''
        if 851 <= float(linha['PRODUTO'][:6]) <= 892:
            return str(linha['PRODUTO'])[:3]
        else:
            return str(linha['PRODUTO'])[:6]

    def extrair_fase(self):
        '''Aqui entende-se fase por etapa do processo, exemplo: corte, prensa...
        Esse método está associado ao fluxo de uma peça. Cada fluxo é escrito em um padrão, onde cada fase está entre ()'''

        padrao = r'\((\d+)\)'
        # Extrair o número e mapear para o nome de cada fase de acordo com o dicionário dicionario_fases
        fases_numerico = self.imol_data.df['FLUXO'].astype(str).apply(lambda x: re.findall(padrao, x))
        fases_texto = []
        for numeros in fases_numerico:
            fases_texto.append([dicionario_fases.get(int(numero), 'Número não encontrado') for numero in numeros])


        df_fases = pd.DataFrame({'FLUXO': self.imol_data.df['FLUXO'], 'FASE_NUMERO': fases_numerico, 'FASE_NOME': fases_texto})
        return df_fases

    def extrair_espessura_material(self, input_string):
        '''Seguindo que para o material a espessura sempre vem antes de MM vamos procurar essa informação usando regex'''
        match = re.search(r'(\d+(\.\d+)?)MM', input_string)
        if match:
            numero_anterior = match.group(1)  # Obtém o número encontrado
            return numero_anterior
        else:
            return -1  # Se não encontrar "MM", retornamos que não há informação sobre a espessura

    def extrair_espessura_descricao(self, descricao):
        '''Seguindo que para a descrição a espessura sempre vem no formato largXcompXespessura vamos procurar essa informação usando regex'''
        # Verifica se a descrição contém um padrão de dimensões com decimais
        match = re.search(r'(\d+(\.\d+)?)\s?[Xx]\s?(\d+(\.\d+)?)\s?[Xx]\s?(\d+(\.\d+)?)', descricao)
        if match:
            return match.group(5)  # Retorna o último grupo correspondente à espessura
        else:
            return -1

    def extrair_nome_material(self, cod_mat):
        '''Utilizando o pdf disponibilizado vamos retirar a informação do nome do material de acordo com o seu código. Essa informação facilitará na análise e podera ser usada para verificar se há descumprimento de alguma restrição'''
        return dicionario_materiais.get(cod_mat, "Material não encontrado")

    def ajusta_texto(self, texto):
        texto_limpo = texto.replace('-', '')
        palavras = texto_limpo.split()
        palavras_limpas = [palavra for palavra in palavras if len(palavra) > 2 or len(palavras) == 1]
        descricao_limpa = ' '.join(palavras_limpas)

        return descricao_limpa



# ----------------------------------------------------------------- # função para pegar o dataframe após o preprocessamento
    def get_df(self):
        return self.imol_data.df

In [4]:
class Restricoes:
    '''Essa classe tem por objetivo verificar se alguma peça que passou ou irá passar por uma determinada máquina descumpre com as restrições da mesma. Sabemos que para toda máquina existem restrições de dois tipos:
            Restrições de dimensão: (mínimo de comprimento, maximo de largura, range de espessura...)
            Restrições de material: (material inadequado a máquina)

    Para padronização iremos adotar o seguinte processo:
        Existira um dcumento/dicionário geral que armazena as restrições de dimensão e material de cada máquina, de maneira padronizada, para que a verificação fique o mais simples possível.'''
    def __init__(self, imol_data):
        self.imol_data = imol_data
        self.df = imol_data.df



# ----------------------------------------------------------------- # 
    def adicionar_nomes_fluxo(self):
        self.imol_data.df['FLUXO_MAQUINAS'] = self.imol_data.df['FLUXO'].apply(self.extrair_nomes_fluxo)

    def verificar_restricoes(self):
        for index, row in self.imol_data.df.iterrows():
            fluxo_maquinas = row['FLUXO_MAQUINAS']
            descumprimentos = []
            motivos = []
            for maquina in fluxo_maquinas:
                if maquina in maquinas_com_restricao:
                    restricoes_maquina = restricoes[maquina]
                    violacao, motivo = self.verificar_restricoes_peca(row, restricoes_maquina)
                    if violacao:
                        descumprimentos.append(maquina)
                        motivos.append(f"{maquina}: {motivo}")
            self.imol_data.df.at[index, 'DESCUMPRIMENTO'] = ', '.join(descumprimentos) if descumprimentos else None
            self.imol_data.df.at[index, 'MOTIVO_DESCUMPRIMENTOS'] = ', '.join(motivos) if motivos else None

    def verificar_restricoes_peca(self, peca, restricoes_maquina):
        # Pegar o menor valor entre LARG e COMP
        menor_dimensao = min(float(peca['LARG']), float(peca['COMP']))

        # Verificar contra dimensão mínima
        if restricoes_maquina.get('dimensao_minima') is not None and menor_dimensao < restricoes_maquina['dimensao_minima']:
            return True, f'Dimensão mínima violada: {menor_dimensao} é menor que o mínimo permitido'

        # Verificar contra dimensão máxima
        if restricoes_maquina.get('dimensao_maxima') is not None and menor_dimensao > restricoes_maquina['dimensao_maxima']:
            return True, f'Dimensão máxima violada: {menor_dimensao} é maior que o máximo permitido'

        # Verificar materiais
        material_nome = peca['MATERIAL_NOME'].lower() # Uniformiza o case para comparação
        materiais_proibidos = restricoes_maquina.get('materiais_proibidos', []) # Garante uma lista vazia como padrão
        if material_nome in [material.lower() for material in materiais_proibidos]: # Compara em case-insensitive
            return True, 'Material incondizente'

        # Verificar tipos de materiais permitidos e proibidos
        tipos = peca['MATERIAL'].split()
        tipo_materiais_proibidos = restricoes_maquina.get('tipo_materiais_proibidos', []) # Garante uma lista vazia como padrão
        for tipo in tipos:
            if tipo.lower() in [tipo_proibido.lower() for tipo_proibido in tipo_materiais_proibidos]: # Compara em case-insensitive
                return True, 'Tipo de material incondizente'

        return False, ''

    def extrair_nomes_fluxo(self, linha):
        linha_sem_parenteses = re.sub(r'\([^)]*\)', ',', linha)
        linha_sem_hifen = linha_sem_parenteses.replace('-', ',')
        return [nome.strip().lower() for nome in linha_sem_hifen.split(',') if nome.strip()]

In [5]:
class Encoding:
    '''Essa classe tem por objetivo codificar as variáveis categóricas do dataset'''
    def __init__(self, imol_data):
        self.imol_data = imol_data
        self.df = imol_data.df

    

In [6]:
class Imol_data:
    def __init__(self, dataframe):
        self.df = dataframe
        self.Restricoes = Restricoes(self)
        self.Preprocessamento = Preprocessamento(self)
        self.Encoding = Encoding(self)

In [7]:
data = pd.read_excel('df_concat.xlsx')

data

,Unnamed: 0,i,PRODUTO,DESCRIÇÃO,QTD,MATERIAL,COMP,LARG,FITACAO,FLUXO,Soma de Contagem
0,0,800.02.00000338- M512 - A - ARM. ALTO E GABIN....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,850.96.00004444,PAC02 - LATERAIS,1.0,- -,-,-,NÃO,(70)EMB01,1.0
2,2,2,852.70.00000500,LATERAL DIR,1.0,- -,-,-,1C-00332,(10)NES01-(20)COL01-(26)CNC01,1.0
3,3,6,852.70.00000501,LATERAL ESQ,1.0,100.10.00009 - CHAPA MDF 2FC BRANCO TX 18MM ...,1900,430,1C-00332,(10)NES01-(20)COL01,1.0
4,4,11,850.96.00004445,PAC01 - TETO-BASE,1.0,- -,-,-,NÃO,(70)EMB01,1.0
...,...,...,...,...,...,...,...,...,...,...,...
23035,23035,1642,853.37.00002525,GAV FRENTE,1.0,100.60.00166 - CHAPA MDP 2FC BETON MATT 18MM ...,373.5,706,2C 2L-06071,(10)NES01-(20)COL01-(26)CNC01,1.0
23036,23036,1647,853.37.00002526,GAV C-FUNDO,1.0,100.60.00165 - CHAPA MDP 2FC BETON MATT 15MM ...,222.5,634,NÃO,(10)NES01-(20)COL01,1.0
23037,23037,1652,853.37.00002527,GAV BASE,1.0,100.60.00165 - CHAPA MDP 2FC BETON MATT 15MM ...,440,655,NÃO,(10)NES01,1.0
23038,23038,1654,853.37.00002534,LATERAL DIR,1.0,100.60.00166 - CHAPA MDP 2FC BETON MATT 18MM ...,760,660,2C-06071,(10)NES01-(20)COL01,1.0


In [8]:
Imol_data = Imol_data(data)

In [9]:
Imol_data.Preprocessamento.tratar()
Imol_data.Restricoes.adicionar_nomes_fluxo()
Imol_data.Restricoes.verificar_restricoes()
df = Imol_data.df

In [12]:
df.to_excel('df_concat_EDA.xlsx', index=False)

In [10]:
"""
maquinas_com_restricao = [
    'esq01', 'esq02', 'esq03', 'esq04',
    'nes01',
    'sec01', 'sec02',
    'ser01', 'ser02',
    'col01', 'col02', 'col03',
    'mob01', 'mob02',
    'pre01', 'pre02',
    'maq01',
    'cnc01',
    'fur01', 'fur02', 'fur03',
    'tup01',
    'cab01',
    'emb01'
]

df_dict = {}  
total = 0

for maquina in maquinas_com_restricao:
    if ['DESCUMPRIMENTO'] is not None:
        df_maquina = df[df['DESCUMPRIMENTO'].str.contains(maquina, case=False, na=False)]
        num_registros = df_maquina.shape[0]
        df_dict[maquina] = df_maquina
        print(f'{maquina}: {num_registros}')
        total += num_registros
        
    else:
        print(f'{maquina}: 0')

print(f'Total: {total}')
"""

"\nmaquinas_com_restricao = [\n    'esq01', 'esq02', 'esq03', 'esq04',\n    'nes01',\n    'sec01', 'sec02',\n    'ser01', 'ser02',\n    'col01', 'col02', 'col03',\n    'mob01', 'mob02',\n    'pre01', 'pre02',\n    'maq01',\n    'cnc01',\n    'fur01', 'fur02', 'fur03',\n    'tup01',\n    'cab01',\n    'emb01'\n]\n\ndf_dict = {}  \ntotal = 0\n\nfor maquina in maquinas_com_restricao:\n    if ['DESCUMPRIMENTO'] is not None:\n        df_maquina = df[df['DESCUMPRIMENTO'].str.contains(maquina, case=False, na=False)]\n        num_registros = df_maquina.shape[0]\n        df_dict[maquina] = df_maquina\n        print(f'{maquina}: {num_registros}')\n        total += num_registros\n        \n    else:\n        print(f'{maquina}: 0')\n\nprint(f'Total: {total}')\n"

In [11]:
"""
import os 

output_directory = r'..\Datasets'

for maquina, df_maquina in df_dict.items():
    if not df_maquina.empty:  
        filename = os.path.join(output_directory, f'{maquina}.xlsx'.replace('/', '_').replace('\\', '_'))
        df_maquina.to_excel(filename, index=False)
        print(f'{maquina}: {df_maquina.shape[0]} registros - Salvo em {filename}')
    else:
        print(f'{maquina}: DataFrame vazio, não salvo')
        
"""

"\nimport os \n\noutput_directory = r'..\\Datasets'\n\nfor maquina, df_maquina in df_dict.items():\n    if not df_maquina.empty:  \n        filename = os.path.join(output_directory, f'{maquina}.xlsx'.replace('/', '_').replace('\\', '_'))\n        df_maquina.to_excel(filename, index=False)\n        print(f'{maquina}: {df_maquina.shape[0]} registros - Salvo em {filename}')\n    else:\n        print(f'{maquina}: DataFrame vazio, não salvo')\n        \n"